In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm
from pymc3.distributions.dist_math import normal_lccdf, normal_lcdf

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=444,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t763be198c21148b7af3ed2a529c0a961" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [3]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [5]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 1.5
𝛽_std = 0.05
𝜓_mean = 0.0
𝜓_std = 0.33
𝜎_std = 0.05

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [6]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,1.536410,0.229932,1.115651
1,0,1.552872,0.026556,0.904660
2,0,1.487890,-0.321123,1.075748
3,0,1.528174,-0.086356,1.094559
4,0,1.503244,0.444541,1.042192


### Generate crossing data

In [7]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

In [8]:
# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

DATA['RI_pooled'] = DATA.error + (DATA.intercept + DATA.b * DATA.dist) / (1 + DATA.b * DATA.dist)
DATA['RI_unpooled'] = DATA.error + (DATA.intercept + DATA.velo * DATA.dist) / (1 + DATA.velo * DATA.dist)
DATA['RI_partpooled'] = DATA.error + (DATA.intercept + DATA.velo_x * DATA.dist) / (1 + DATA.velo_x * DATA.dist)

In [9]:
# censor values in range 0-1
DATA.loc[DATA['RI_pooled'] < 0, 'RI_pooled'] = 0
DATA.loc[DATA['RI_pooled'] > 1, 'RI_pooled'] = 1
DATA.loc[DATA['RI_unpooled'] < 0, 'RI_unpooled'] = 0
DATA.loc[DATA['RI_unpooled'] > 1, 'RI_unpooled'] = 1
DATA.loc[DATA['RI_partpooled'] < 0, 'RI_partpooled'] = 0
DATA.loc[DATA['RI_partpooled'] > 1, 'RI_partpooled'] = 1

In [10]:
DATA

,sidx0,sidx1,dist,b,velo,velo_x,intercept,error,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.014093,1.569123,1.825611,3.589434,0.058843,0.003385,0.082589,0.085834,0.107543
1,0,2,0.093817,1.475324,1.384133,3.666723,0.060876,-0.006604,0.168453,0.162206,0.294644
2,0,3,0.232354,1.524398,1.667974,3.734608,0.063433,-0.013034,0.295365,0.311992,0.485525
3,0,4,0.232354,1.568007,2.242480,3.725850,0.080916,0.064646,0.390995,0.460403,0.572029
4,0,5,0.234463,1.587113,2.505924,3.761738,0.074592,-0.058734,0.266829,0.358349,0.449548
...,...,...,...,...,...,...,...,...,...,...,...
3155,76,78,0.057491,1.515498,2.497094,-0.487928,0.030432,-0.105419,0.002719,0.046731,0.000000
3156,76,79,0.194995,1.430819,1.861327,-0.579221,0.045074,-0.024966,0.228417,0.274402,0.000000
3157,77,78,0.057491,1.455528,2.218625,-0.729714,0.018721,0.122620,0.217113,0.252345,0.098371
3158,77,79,0.194995,1.432789,1.644798,-0.759066,0.117312,0.051167,0.361236,0.382832,0.015131


In [11]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,error,RI_pooled,RI_unpooled,RI_partpooled
0,1,26,0.523417,1.570259,1.690114,3.555288,0.037579,-0.025917,0.445831,0.463415,0.637677
1,33,53,0.555803,1.505988,0.773893,3.082311,0.001843,-0.020690,0.435957,0.281362,0.611415
2,34,63,1.000000,1.564016,1.027060,1.963146,0.065470,-0.015290,0.620231,0.523682,0.669325
3,13,67,1.000000,1.458742,1.518098,1.991522,0.042036,0.031772,0.642156,0.651340,0.711545
4,18,45,0.541446,1.532731,1.904726,3.538284,0.025705,0.091338,0.558905,0.611699,0.757194


### Visualize data

In [12]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Pooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_pooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t4bbe7a93495945b78ea74b39aea68e7c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(65.1%,7.9%,16.1%);fill-opacity:1.0;opacity:0.25;stroke:rgb(0%,0%,0%);stroke-opacity:1.0

In [13]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Unpooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_unpooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te2d9a66afa2e4906a08f388951078df6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(71.7%,13.3%,18.7%);fill-opacity:1.0;opacity:0.25;stroke:rgb(0%,0%,0%);stroke-opacity:1.0" tra

In [14]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Partpooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_partpooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te3d09859c0814e7db360db98a37bb889" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g

### Funcrions to plot results

In [16]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [17]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

### Define models

In [22]:
def censored_pooled_regression(x, y, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5.0, shape=1)

        # linear model prediction
        ri = (𝛼 + 𝛽 * _x) / (1 + 𝛽 * _x)

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf((𝛼 + 𝛽 * _x_lc) / (1 + 𝛽 * _x_lc), 𝜎, _y_lc),
            )
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf((𝛼 + 𝛽 * _x_uc) / (1 + 𝛽 * _x_uc), 𝜎, _y_uc),
            )

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [25]:
def censored_unpooled_noncentered_regression(x, y, idx0, idx1, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:
        
        # censored indexers
        sidx0 = pm.Data("spp_idx0", idx0.values[uncensored])
        sidx1 = pm.Data("spp_idx1", idx1.values[uncensored])
        sidx0_u = pm.Data("sidx0_u", idx0.values[upper_censored])
        sidx1_u = pm.Data("sidx1_u", idx1.values[upper_censored])
        sidx0_l = pm.Data("sidx0_l", idx0.values[lower_censored])
        sidx1_l = pm.Data("sidx1_l", idx1.values[lower_censored])

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5., shape=1)
        
        # linear model prediction
        ri = (𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x) / (1 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x)

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf((𝛼 + (𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc) / (1 + (𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc), 
                            𝜎, _y_lc),
            )
        
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf((𝛼 + (𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc) / (1 + (𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc), 
                            𝜎, _y_uc),
            )
            

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [26]:
def censored_partpooled_noncentered_regression(x, y, idx0, idx1, gidx, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:
        
        # censored indexers
        sidx0 = pm.Data("spp_idx0", idx0.values[uncensored])
        sidx1 = pm.Data("spp_idx1", idx1.values[uncensored])
        sidx0_u = pm.Data("sidx0_u", idx0.values[upper_censored])
        sidx1_u = pm.Data("sidx1_u", idx1.values[upper_censored])
        sidx0_l = pm.Data("sidx0_l", idx0.values[lower_censored])
        sidx1_l = pm.Data("sidx1_l", idx1.values[lower_censored])
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5., shape=1)
        
        # linear model prediction
        ri = (𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x) / (1 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x)

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf((𝛼 + (𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc) / (1 + (𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc), 
                            𝜎, _y_lc),
            )
        
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf((𝛼 + (𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc) / (1 + (𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc), 
                            𝜎, _y_uc),
            )

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Run models

In [27]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=4000,
    draws=4000,
    target_accept=0.95,
    return_inferencedata=False,
    progressbar=True,
)

In [28]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_sub = censored_pooled_regression(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛽, 𝛼]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 46 seconds.


In [29]:
pooled_sub['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝛼[0],0.054,0.010,0.035,0.071,0.000,0.0,4214.0,4193.0,4220.0,4470.0,1.0
𝛽[0],1.473,0.034,1.408,1.535,0.001,0.0,4222.0,4220.0,4231.0,4632.0,1.0
𝜎[0],0.052,0.001,0.050,0.054,0.000,0.0,5753.0,5708.0,5773.0,5008.0,1.0


In [30]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_sub = censored_unpooled_noncentered_regression(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 4405 seconds.
There were 3070 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 1153 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 360 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 3450 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effe

In [31]:
unpooled_sub['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],-0.795,3.310,-6.404,6.060,1.256,0.928,7.0,7.0,7.0,24.0,1.56
𝜓_offset[0],0.130,0.375,-0.587,0.490,0.185,0.141,4.0,4.0,5.0,24.0,2.21
𝜓_offset[1],0.090,0.709,-0.647,1.295,0.353,0.270,4.0,4.0,5.0,19.0,2.64
𝜓_offset[2],-0.082,0.518,-0.731,0.803,0.256,0.196,4.0,4.0,4.0,15.0,3.50
𝜓_offset[3],0.649,0.403,0.073,1.189,0.196,0.150,4.0,4.0,5.0,29.0,2.49
...,...,...,...,...,...,...,...,...,...,...,...
𝜓[76],-0.853,4.584,-8.095,7.054,2.031,1.527,5.0,5.0,5.0,14.0,2.07
𝜓[77],-1.651,5.833,-12.922,6.612,2.695,2.039,5.0,5.0,5.0,14.0,2.46
𝜓[78],2.986,3.788,-5.302,9.020,1.389,1.024,7.0,7.0,8.0,48.0,1.49
𝜓[79],-0.668,4.251,-7.778,6.278,1.840,1.380,5.0,5.0,6.0,16.0,1.85


In [32]:
toytrace(unpooled_sub['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t180be9283fef42f3bc8a3c780bb97347" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 149.43925233644862 L 53.72766937572699 150.0 L 57.45533875145401 149.43925233644862 L 61.183008127181 149.06542056074767 L 64.91067750290799 149.81308411214954 L 68.63834687863498 149.81308411214954 L 72.366016254362 149.62616822429905 L 76.09368563008897 147.7570093457944 L 79.82135500581597 147.38317757009347 L 83.54902438154295 147.38317757009347 L 87.27669375726998 145.3271028037383 L 91.00436313299697 147.38317757009347 L 94.73203250872396 145.88785046728972 L 98.45970188445094 144.2056074766355 L 102.18737126017795 145.3271028037383 L 105.91504063590494 142.33644859813083 L 109.64271001163193 141.77570093457945 L 113.37037938735892 144.57943925233644 L 117.09804876308593 143.45794392523365 L 120.82571813881293 144.7663551401869 L 124.55338751453993 142.52336448598132 L 128.2810568902669 130.18691588785046 L 132.00872626599391 55.794392523364486 L 135.7363956417209 129.25233644859813 L 139.4640650174479 138.03738317757012 L 143.1917343931749 140.65420560747663 L 146.9194037689019 145.3271028037383 L 150.6470731446289 142.52336448598132 L 154.3747425203559 132.99065420560748 L 158.10241189608286 139.34579439252337 L 161.8300812718099 134.6728971962617 L 165.55775064753686 134.48598130841123 L 169.28542002326384 127.94392523364488 L 173.01308939899087 132.42990654205607 L 176.74075877471785 130.93457943925233 L 180.46842815044485 131.30841121495328 L 184.19609752617183 125.70093457943925 L 187.92376690189883 121.21495327102804 L 191.65143627762583 110.93457943925233 L 195.37910565335284 121.96261682242991 L 199.1067750290798 116.54205607476635 L 202.83444440480682 111.86915887850468 L 206.56211378053382 100.09345794392523 L 210.2897831562608 99.53271028037383 L 214.01745253198783 89.06542056074767 L 217.74512190771483 50.0 L 221.47279128344178 66.26168224299064 L 225.20046065916878 67.19626168224298 L 228.9281300348958 94.67289719626169 L 232.65579941062282 92.2429906542056 L 236.3834687863498 70.37383177570094 L 240.11113816207677 102.71028037383178 L 243.83880753780375 87.75700934579439 L 247.56647691353078 87.38317757009347 L 251.29414628925778 78.78504672897196 L 255.02181566498473 90.37383177570094 L 258.74948504071176 104.01869158878506 L 262.4771544164388 107.7570093457944 L 266.20482379216577 112.99065420560747 L 269.93249316789274 117.66355140186917 L 273.6601625436197 123.08411214953271 L 277.38783191934675 126.07476635514018 L 281.1155012950737 126.44859813084112 L 284.84317067080076 130.74766355140187 L 288.5708400465277 123.64485981308411 L 292.2985094222547 123.83177570093459 L 296.02617879798174 121.21495327102804 L 299.7538481737087 127.38317757009345 L 303.4815175494357 118.41121495327101 L 307.2091869251627 125.70093457943925 L 310.9368563008897 128.69158878504672 L 314.6645256766167 123.64485981308411 L 318.39219505234365 126.63551401869158 L 322.1198644280707 129.06542056074767 L 325.84753380379766 136.91588785046727 L 329.5752031795247 137.10280373831776 L 333.30287255525167 131.49532710280374 L 337.03054193097864 135.42056074766356 L 340.7582113067057 132.99065420560748 L 344.48588068243265 131.30841121495328 L 348.2135500581596 140.46728971962614 L 351.94121943388666 139.53271028037383 L 355.6688888096136 133.73831775700936 L 359.39655818534067 136.16822429906543 L 363.12422756106764 133.17757009345794 L 366.8518969367946 140.65420560747663 L 370.5795663125216 137.85046728971963 L 374.3072356882486 135.79439252336448 L 378.03490506397554 143.2710280

In [33]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    unpooled_sub['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t92ebd0678ee94262a8712fceae1cd237" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -5 0 5 ESTIMATED species velocity -1.0 -0.5 0.0 0.5 TRUE species velocity

In [34]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_sub = censored_partpooled_noncentered_regression(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 3070 seconds.
There were 3998 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2603 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 3982 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3984 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [35]:
toytrace(partpooled_sub['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="tbf7bd9678bd8483d8af688be106711ab" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 266.2019178835215 149.90033222591362 L 267.3734464171974 149.468438538206 L 268.5449749508732 149.26910299003322 L 269.71650348454904 146.44518272425248 L 270.8880320182249 147.74086378737542 L 272.05956055190074 146.17940199335547 L 273.23108908557657 144.65116279069767 L 274.40261761925245 143.52159468438538 L 275.5741461529283 148.13953488372093 L 276.74567468660405 146.67774086378736 L 277.91720322027993 146.47840531561462 L 279.08873175395576 148.57142857142858 L 280.2602602876316 145.3156146179402 L 281.43178882130746 145.68106312292358 L 282.60331735498335 143.78737541528238 L 283.7748458886592 147.27574750830564 L 284.94637442233505 147.8405315614618 L 286.1179029560109 147.87375415282395 L 287.28943148968665 147.30897009966776 L 288.46096002336253 147.3421926910299 L 289.63248855703836 147.0764119601329 L 290.8040170907142 146.91029900332225 L 291.97554562439 147.14285714285714 L 293.14707415806583 146.64451827242524 L 294.3186026917417 143.98671096345515 L 295.4901312254176 145.51495016611295 L 296.6616597590934 146.67774086378736 L 297.83318829276925 147.6079734219269 L 299.0047168264451 149.06976744186048 L 300.17624536012096 149.2358803986711 L 301.3477738937968 148.63787375415282 L 302.5193024274726 148.93687707641195 L 303.69083096114844 149.1029900332226 L 304.8623594948243 148.93687707641195 L 306.03388802850014 149.03654485049836 L 307.20541656217597 149.3687707641196 L 308.37694509585185 148.90365448504983 L 309.5484736295277 149.40199335548172 L 310.7200021632035 149.83388704318938 L 311.89153069687933 149.80066445182726 L 313.06305923055515 149.468438538206 L 314.234587764231 149.56810631229237 L 315.40611629790686 150.0 L 316.57764483158275 150.0 L 317.74917336525857 150.0 L 318.9207018989344 150.0 L 320.0922304326102 150.0 L 321.2637589662861 150.0 L 322.435287499962 150.0 L 323.60681603363776 150.0 L 324.7783445673136 150.0 L 325.94987310098946 150.0 L 327.1214016346653 150.0 L 328.2929301683411 150.0 L 329.46445870201694 150.0 L 330.63598723569277 150.0 L 331.80751576936865 150.0 L 332.9790443030445 150.0 L 334.15057283672036 148.37209302325581 L 335.3221013703962 147.67441860465115 L 336.49362990407207 143.22259136212625 L 337.66515843774783 144.28571428571428 L 338.8366869714237 147.87375415282395 L 340.00821550509954 143.58803986710964 L 341.17974403877537 140.1328903654485 L 342.3512725724512 132.82392026578074 L 343.5228011061271 139.5016611295681 L 344.69432963980285 133.95348837209303 L 345.8658581734787 133.1893687707641 L 347.0373867071546 145.71428571428572 L 348.20891524083044 148.17275747508305 L 349.3804437745063 88.23920265780731 L 350.55197230818214 121.32890365448505 L 351.72350084185797 142.79069767441862 L 352.8950293755338 149.80066445182726 L 354.0665579092097 150.0 L 355.23808644288545 150.0 L 356.4096149765613 150.0 L 357.58114351023715 150.0 L 358.752672043913 148.57142857142858 L 359.92420057758886 144.08637873754154 L 361.0957291112647 143.05647840531563 L 362.2672576449405 145.14950166112956 L 363.43878617861634 143.82059800664453 L 364.6103147122922 144.05315614617942 L 365.7818432459681 147.5747508305648 L 366.9533717796439 144.21926910299004 L 368.1249003133198 146.8770764119601 L 369.2964288469956 142.2923588039867 L 370.4679573806714 144.6843853820598 L 371.63948591434723 144.11960132890368 L 372.8110144480231 142.0265780730897 L 373.982542981699 141.82724252491695 L 375.1540715153748 149.30232558139534 L 376.32560004905065 

In [36]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_sub['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="te1a60edd41cc46dc93dcc1dea284738d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -5 0 5 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [45]:
draw_velocity_dists(partpooled_sub['trace'], baseline = 0.1);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t4e72586b43614422a41e74f10e1f3e66" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="72.99260141110977,250.0 75.12766281095941,250.0 77.26272421080907,250.0 79.39778561065872,250.0 81.53284701050836,250.0 83.66790841035802,250.0 85.80296981020766,250.0 87.93803121005732,250.0 90.07309260990695,250.0 92.20815400975661,250.0 94.34321540960624,250.0 96.47827680945589,250.0 98.61333820930554,250.0 100.74839960915519,250.0 102.88346100900483,250.0 105.01852240885448,250.0 107.15358380870413,250.0 109.28864520855379,250.0 111.42370660840342,250.0 113.55876800825307,250.0 115.69382940810273,250.0 117.82889080795236,250.0 119.96395220780201,250.0 122.09901360765166,250.0 124.23407500750132,250.0 126.36913640735096,250.0 128.5041978072006,250.0 130.63925920705026,250.0 132.7743206068999,250.0 134.90938200674955,250.0 137.04444340659919,250.0 139.17950480644885,250.0 141.31456620629848,250.0 143.44962760614814,250.0 145.58468900599777,250.0 147.71975040584744,250.0 149.85481180569707,250.0 151.98987320554673,250.0 154.12493460539636,250.0 156.25999600524602,250.0 158.39505740509568,250.0 160.53011880494532,250.0 162.66518020479495,250.0 164.8002416046446,250.0 166.93530300449424,250.0 169.0703644043439,250.0 171.20542580419357,250.0 173.3404872040432,250.0 175.47554860389286,250.0 177.6106100037425,250.0 179.74567140359216,250.0 181.8807328034418,250.0 184.01579420329145,250.0 186.15085560314108,250.0 188.28591700299074,250.0 190.42097840284038,250.0 192.55603980269004,250.0 194.69110120253967,250.0 196.82616260238933,250.0 198.96122400223896,250.0 201.09628540208863,250.0 203.23134680193826,250.0 205.36640820178786,250.0 207.50146960163758,250.0 209.6365310014872,250.0 211.77159240133682,250.0 213.90665380118654,250.0 216.04171520103614,250.0 218.1767766008858,250.0 220.31183800073546,250.0 222.4468994005851,250.0 224.5819608004347,250.0 226.71702220028442,250.0 228.85208360013402,250.0 230.98714499998363,250.0 233.12220639983332,250.0 235.25726779968298,250.0 237.39232919953264,250.0 239.52739059938227,250.0 241.66245199923188,250.0 243.7975133990816,250.0 245.9325747989312,250.0 248.0676361987808,250.0 250.20269759863052,250.0 252.33775899848015,250.0 254.47282039832976,250.0 256.60788179817945,250.0 258.7429431980291,250.0 260.8780045978787,250.0 263.0130659977284,250.0 265.148127397578,250.0 267.2831887974276,250.0 269.41825019727736,250.0 271.553311597127,250.0 273.6883729969766,250.0 275.82343439682626,250.0 277.9584957966759,250.0 280.0935571965255,250.0 282.2286185963752,250.0 284.36367999622485,250.0 284.36367999622485,249.00320571053476 282.2286185963752,248.83308696212563 280.0935571965255,248.63832064709206 277.9584957966759,248.4161479115779 275.82343439682626,248.1636401027026 273.6883729969766,247.87771166210328 271.553311597127,247.55513780907714 269.41825019727736,247.19257740919699 267.2831887974276,246.7866013613483 265.148127397578,246.33372675425713 263.0130659977284,245.83045694238155 260.8780045978787,245.27332757075163 258.7429431980291,244.65895843989216 256.60788179817945,243.98411094701945 254.47282039832976,243.24575067075406 252.33775899848015,242.44111448692055 250.20269759863052,241.56778141671433 248.0676361987808,240.6237462204626 245.9325747989312,239.6074945659087 243.7975133990816,238.51807842548618 241.66245199923188,237.35519019888528 239.52739059938227,236.11923392200188 237.39232919953264,234.81139181772787 235.25726779968298,233.43368437433713 233.12220639983332,231.98902210924345 230.98714499998363,230.48124719464494 228.852

### Assess model fit

In [38]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val                                           #get the ^

In [39]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [40]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

2.410852838638037

In [41]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

2.537470515573881

In [42]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-68.48522152338049

In [43]:
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-41.54120542064919